# Infeasibility UQ

Take the generic DEMO solution, turned into an input file. Remove f-values at iteration vars, and replace their equality constraints with inequalities. Run PROCESS once-through with uncertain inputs, and the QoI as the value of constraints, i.e. the infeasibility.

Dask is used to parallelise the evaluations for a SLURM cluster.


In [1]:
import easyvvuq as uq
import chaospy as cp
from pathlib import Path
from dask.distributed import Client
from dask_jobqueue import SLURMCluster

## Campaign to capture feasibility

Using the epistemic uncertain inputs for the entire code, capture the distribution of constraint residuals.

To start with, make just 4 inputs uncertain.


In [2]:
# Init cluster (describes a single node, or less if need less than that per worker)
# cluster = SLURMCluster(
#     cores=56,
#     processes=4,  # check docs
#     memory="192GB",
#     account="UKAEA-AP002-CPU",
#     walltime="01:00:00",
#     queue="cclake",
# )

# Need less than a full node per worker
cluster = SLURMCluster(
    cores=1,
    processes=1,
    memory="4GB",
    account="UKAEA-AP002-CPU",
    walltime="00:05:00",
    queue="cclake",
)
cluster.scale(4)  # 4 workers
# print(cluster.job_script())

# Connect Dask client to remote cluster
client = Client(cluster)
# Code from now on submitted to batch queue

# Define campaign
WORK_DIR = "campaigns"
Path("campaigns").mkdir(exist_ok=True)
campaign = uq.Campaign(name="example_cluster", work_dir=WORK_DIR)

# Define parameter space
# Uncertainties from Alex's SA paper

params = {
    "fdene": {
        "type": "float",
        "min": 1.1,
        "max": 1.3,
        "default": 1.2,
    },  # check: not sure if this is right var. I think ok
    "hfact": {"type": "float", "min": 1.0, "max": 1.2, "default": 1.2},
    "coreradius": {"type": "float", "min": 0.45, "max": 0.75, "default": 0.75},  # ok
    "fimp_2": {"type": "float", "min": 0.085, "max": 0.115, "default": 0.1},  # ok
    "fimp_14": {
        "type": "float",
        "min": 1.0e-5,
        "max": 1.0e-4,
        "default": 1.0e-5,
    },  # ok
    "psepbqarmax": {"type": "float", "min": 8.7, "max": 9.7, "default": 9.0},  # ok
    "flhthresh": {"type": "float", "min": 0.85, "max": 1.15, "default": 1.15},  # ok
    "cboot": {
        "type": "float",
        "min": 0.95,
        "max": 1.05,
        "default": 1.0,
    },  # ok
    "peakfactrad": {"type": "float", "min": 2.0, "max": 3.5, "default": 3.33},  # ok
    "kappa": {"type": "float", "min": 1.8, "max": 1.9, "default": 1.848},  # ok
    "etaech": {"type": "float", "min": 0.3, "max": 0.5, "default": 0.4},  # ok
    "feffcd": {"type": "float", "min": 0.5, "max": 5.0, "default": 1.0},  # ok
    "etath": {"type": "float", "min": 0.36, "max": 0.4, "default": 0.375},  # ok
    "etaiso": {"type": "float", "min": 0.75, "max": 0.95, "default": 0.9},  # ok
    "boundl_18": {
        "type": "float",
        "min": 3.25,
        "max": 3.75,
        "default": 3.25,
    },  # q^95_min, ok
    "pinjalw": {"type": "float", "min": 51.0, "max": 61.0, "default": 61.0},  # ok
    "alstroh": {"type": "float", "min": 6.0e8, "max": 7.2e8, "default": 6.6e8},  # ok
    "sig_tf_wp_max": {
        "type": "float",
        "min": 5.2e8,
        "max": 6.4e8,
        "default": 6.4e8,
    },  # ok, but might need sig_tf_case_max to be the same too
    "aspect": {"type": "float", "min": 3.0, "max": 3.2, "default": 3.1},
    "boundu_2": {
        "type": "float",
        "min": 11.0,
        "max": 12.0,
        "default": 12.0,
    },  # B_T^max, ok
    "triang": {"type": "float", "min": 0.4, "max": 0.6, "default": 0.5},  # ok
    "vary_param": {
        "type": "string",
        "default": "",
    },  # param being changed: used for analysis only
}

# QoIs
# Violated constraint residuals
qois = [
    "coheof","bore","powfmw"
]

# Create encoder and decoder
encoder = uq.encoders.GenericEncoder(
    template_fname="demo_sol_max_net_elec_no_f_IN.template", target_filename="IN.DAT"
)
decoder = uq.decoders.JSONDecoder(target_filename="MFILE.DAT.json", output_columns=qois)

cmd = "process -i IN.DAT --plot --mfile MFILE.DAT"
actions = uq.actions.local_execute(encoder, cmd, decoder)

# Add the app
campaign.add_app(name="feasibility", params=params, actions=actions)

# Create PCE sampler, 4 uncertainties
vary = {
    "aspect": cp.Uniform(3.0, 3.2),
    "triang": cp.Uniform(0.4, 0.6),
    "psepbqarmax": cp.Uniform(8.7, 9.7),
    "hfact": cp.Uniform(1.0, 1.2),
}
pce_sampler = uq.sampling.MCSampler(vary=vary,n_mc_samples=5)

# Add pce_sampler to campaign
campaign.set_sampler(pce_sampler)

# Draw samples, execute and collate
campaign.execute(pool=client).collate(progress_bar=True)
samples = campaign.get_collation_result()
samples


  0%|          | 0/300 [00:00<?, ?it/s]


 29%|██▉       | 87/300 [00:00<00:00, 867.72it/s]


 74%|███████▍  | 222/300 [00:00<00:00, 1149.87it/s]


100%|██████████| 300/300 [00:00<00:00, 1184.70it/s]

,run_id,iteration,aspect,triang,psepbqarmax,hfact,fdene,coreradius,fimp_2,fimp_14,...,etaiso,boundl_18,pinjalw,alstroh,sig_tf_wp_max,boundu_2,vary_param,coheof,bore,powfmw
,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0,1,0,3.192460,0.564947,9.675370,1.008353,1.2,0.75,0.1,0.00001,...,0.9,3.25,61.0,660000000.0,640000000.0,12.0,,9102000.0,2.8236,2443.8
1,2,0,3.089958,0.564947,9.675370,1.008353,1.2,0.75,0.1,0.00001,...,0.9,3.25,61.0,660000000.0,640000000.0,12.0,,9102000.0,2.8236,2591.9
2,3,0,3.192460,0.436303,9.675370,1.008353,1.2,0.75,0.1,0.00001,...,0.9,3.25,61.0,660000000.0,640000000.0,12.0,,9102000.0,2.8236,2504.1
3,4,0,3.192460,0.564947,8.837430,1.008353,1.2,0.75,0.1,0.00001,...,0.9,3.25,61.0,660000000.0,640000000.0,12.0,,9102000.0,2.8236,2443.8
4,5,0,3.192460,0.564947,9.675370,1.077303,1.2,0.75,0.1,0.00001,...,0.9,3.25,61.0,660000000.0,640000000.0,12.0,,9102000.0,2.8236,2443.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,296,0,3.186025,0.453899,9.022908,1.031454,1.2,0.75,0.1,0.00001,...,0.9,3.25,61.0,660000000.0,640000000.0,12.0,,9102000.0,2.8236,2505.3
296,297,0,3.002216,0.541700,9.022908,1.031454,1.2,0.75,0.1,0.00001,...,0.9,3.25,61.0,660000000.0,640000000.0,12.0,,9102000.0,2.8236,2741.5
297,298,0,3.002216,0.453899,9.590169,1.031454,1.2,0.75,0.1,0.00001,...,0.9,3.25,61.0,660000000.0,640000000.0,12.0,,9102000.0,2.8236,2789.5
